In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy

# Disuguaglianza di CHSH

*Stima d'uso: Due minuti su un processore Heron r2 (NOTA: Questa è solo una stima. Il tempo di esecuzione effettivo potrebbe variare.)*
## Contesto
In questo tutorial, eseguirai un esperimento su un computer quantistico per dimostrare la violazione della disuguaglianza di CHSH con la primitiva Estimator.

La disuguaglianza di CHSH, che prende il nome dagli autori Clauser, Horne, Shimony e Holt, viene utilizzata per dimostrare sperimentalmente il teorema di Bell (1969). Questo teorema afferma che le teorie a variabili nascoste locali non possono spiegare alcune conseguenze dell'entanglement nella meccanica quantistica. La violazione della disuguaglianza di CHSH viene utilizzata per dimostrare che la meccanica quantistica è incompatibile con le teorie a variabili nascoste locali. Questo è un esperimento importante per comprendere i fondamenti della meccanica quantistica.

Il Premio Nobel per la Fisica del 2022 è stato assegnato ad Alain Aspect, John Clauser e Anton Zeilinger in parte per il loro lavoro pionieristico nella scienza dell'informazione quantistica e, in particolare, per i loro esperimenti con fotoni entangled che dimostrano la violazione delle disuguaglianze di Bell.
## Requisiti
Prima di iniziare questo tutorial, assicurati di avere installato quanto segue:

* Qiskit SDK v1.0 o successivo, con supporto per la [visualizzazione](https://docs.quantum.ibm.com/api/qiskit/visualization)
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 o successivo
## Configurazione

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## Passo 1: Mappare gli input classici su un problema quantistico
Per questo esperimento, creeremo una coppia entangled su cui misureremo ogni qubit in due basi diverse. Etichetteremo le basi per il primo qubit come $A$ e $a$ e le basi per il secondo qubit come $B$ e $b$. Questo ci consente di calcolare la quantità CHSH $S_1$:

$$
S_1 = A(B-b) + a(B+b).
$$

Ogni osservabile è $+1$ oppure $-1$. Chiaramente, uno dei termini $B\pm b$ deve essere $0$ e l'altro deve essere $\pm 2$. Pertanto, $S_1 = \pm 2$. Il valore medio di $S_1$ deve soddisfare la disuguaglianza:

$$
|\langle S_1 \rangle|\leq 2.
$$

Espandendo $S_1$ in termini di $A$, $a$, $B$ e $b$ si ottiene:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Puoi definire un'altra quantità CHSH $S_2$:

$$
S_2 = A(B+b) - a(B-b),
$$

Questo porta a un'altra disuguaglianza:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Se la meccanica quantistica può essere descritta da teorie a variabili nascoste locali, le disuguaglianze precedenti devono essere valide. Tuttavia, come viene dimostrato in questo tutorial, queste disuguaglianze possono essere violate in un computer quantistico. Pertanto, la meccanica quantistica non è compatibile con le teorie a variabili nascoste locali.
Se desideri approfondire la teoria, esplora [Entanglement in Action](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) con John Watrous.
Creerete una coppia entangled tra due qubit in un computer quantistico creando lo stato di Bell $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$. Utilizzando la primitiva Estimator, potete ottenere direttamente i valori di aspettazione necessari ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$ e $\langle ab \rangle$) per calcolare i valori di aspettazione delle due quantità CHSH $\langle S_1\rangle$ e $\langle S_2\rangle$. Prima dell'introduzione della primitiva Estimator, avresti dovuto costruire i valori di aspettazione dai risultati delle misurazioni.

Misurerete il secondo qubit nelle basi $Z$ e $X$. Il primo qubit verrà misurato anch'esso in basi ortogonali, ma con un angolo rispetto al secondo qubit, che spazieremo tra $0$ e $2\pi$. Come vedrai, la primitiva Estimator rende molto semplice l'esecuzione di circuiti parametrizzati. Invece di creare una serie di circuiti CHSH, devi creare solo *un* circuito CHSH con un parametro che specifica l'angolo di misurazione e una serie di valori di fase per il parametro.

Infine, analizzerai i risultati e li tracerai in funzione dell'angolo di misurazione. Vedrai che per un certo intervallo di angoli di misurazione, i valori di aspettazione delle quantità CHSH $|\langle S_1\rangle| > 2$ o $|\langle S_2\rangle| > 2$, il che dimostra la violazione della disuguaglianza di CHSH.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### Creare un circuito CHSH parametrizzato
Per prima cosa, scriviamo il circuito con il parametro $\theta$, che chiamiamo `theta`. La [primitiva `Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) può semplificare enormemente la costruzione del circuito e l'analisi dell'output fornendo direttamente i valori di aspettazione delle osservabili. Molti problemi di interesse, specialmente per applicazioni a breve termine su sistemi rumorosi, possono essere formulati in termini di valori di aspettazione. La primitiva `Estimator` (V2) può cambiare automaticamente la base di misurazione in base all'osservabile fornita.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### Creare un elenco di valori di fase da assegnare successivamente
Dopo aver creato il circuito CHSH parametrizzato, creerai un elenco di valori di fase da assegnare al circuito nel passo successivo. Puoi utilizzare il seguente codice per creare un elenco di 21 valori di fase compresi tra $0$ e $2 \pi$ con spaziatura uniforme, ovvero $0$, $0.1 \pi$, $0.2 \pi$, ..., $1.9 \pi$, $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### Osservabili
Ora abbiamo bisogno delle osservabili da cui calcolare i valori di aspettazione. Nel nostro caso stiamo considerando basi ortogonali per ogni qubit, lasciando che la rotazione $Y$ parametrizzata per il primo qubit spazzi la base di misurazione quasi continuamente rispetto alla base del secondo qubit. Sceglieremo quindi le osservabili $ZZ$, $ZX$, $XZ$ e $XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## Passo 2: Ottimizzare il problema per l'esecuzione su hardware quantistico

Per ridurre il tempo totale di esecuzione del job, le primitive V2 accettano solo circuiti e osservabili conformi alle istruzioni e alla connettività supportate dal sistema target (chiamati circuiti e osservabili ISA - instruction set architecture).

### Circuito ISA

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### Osservabili ISA

Allo stesso modo, dobbiamo trasformare le osservabili per renderle compatibili con il backend prima di eseguire i job con [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run). Possiamo eseguire la trasformazione utilizzando il metodo `apply_layout` dell'oggetto `SparsePauliOp`.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## Passo 3: Eseguire utilizzando le primitive Qiskit
Per eseguire l'intero esperimento in una singola chiamata all'[`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2).
Possiamo creare una primitiva [Qiskit Runtime `Estimator`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2) per calcolare i nostri valori di aspettazione. Il metodo `EstimatorV2.run()` accetta un iterabile di `primitive unified blocs (PUB)`. Ogni PUB è un iterabile nel formato `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)